### Import packages

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

import json

import txt2xml
import os

latent_size = 60

### Configurations:

In [2]:
file_path = "./generated_levels"

In [3]:
latent_vec_list = []

for i in range(5):
    latent_vec = tf.random.normal(shape=[1, latent_size], mean=i*5, stddev=1)
    latent_vec_list.append(latent_vec)
    
for i in range(5):
    latent_vec = np.random.normal(5*(i+5), 1, latent_size)
    latent_vec = tf.convert_to_tensor(latent_vec)
    latent_vec_list.append(latent_vec)

### Load dictionaries and model:

In [4]:
with open("vocabulary.json") as json_file:
    vocab = json.load(json_file)

with open("vocab_dec.json") as json_file:
    vocab_dec = json.load(json_file)

In [5]:
variational_decoder = tf.keras.models.load_model("./saved_models/variational_decoder.tf")

### Define functions:

In [6]:
def generate_single(latent_vec, variational_decoder = variational_decoder, vocab_dec = vocab_dec, max_length = 30):
    generated = ""
    for idx in range(max_length):
        decoder_inputs = tf.constant(["sos " + generated])
        latent_vec = tf.random.normal(shape=[1, latent_size])
        Y_proba = variational_decoder.predict((latent_vec, decoder_inputs))[0, idx]
        predicted_id = np.argmax(Y_proba)
        predicted_word = vocab_dec[predicted_id]
        if predicted_word == "eos":
            break
        generated += " " + predicted_word
    return generated.strip().split()

In [7]:
def generate(latent_vec_list, file_path = file_path):
    deconverter = txt2xml.txt2xml()
    for i in range(len(latent_vec_list)):
        latent_vec = latent_vec_list[i]
        level = generate_single(latent_vec)
        level_txt = [vocab[int(s)] for s in level]
        level_xml = deconverter.vector2xml(level_txt)
        with open(file_path + "/level-vec-" + str(i) + ".xml", "w") as f:
            f.write(level_xml)
    print("Done")

### Generate:

In [8]:
generate(latent_vec_list)

1/1 [==============================] - 0s 38ms/step
Done
